Imports

In [19]:
import warnings
warnings.filterwarnings('ignore')

# Nifti Import

**From Directory**
___

Here's a brief markup (in Markdown format) that explains the purpose and usage of the `segments_dict`:

---

## Neuroimaging File Extraction Dictionary

The `segments_dict` is a predefined dictionary structured to facilitate the extraction of specific types of neuroimaging files. Each key in the dictionary represents a distinct neuroimaging segment, and its associated value is another dictionary containing the following fields:

- **path**: This should be filled with the absolute path to the base directory containing the neuroimaging files for the corresponding segment. 
- **glob_name_pattern**: This is the string pattern that will be used to "glob" or search for the specific files within the provided path. It helps in identifying and extracting the desired files based on their naming conventions.

Here's a breakdown of the segments and their respective fields:

### 1. Cerebrospinal Fluid (CSF)
- **path**: Absolute path to the base directory containing CSF files.
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **path**: Absolute path to the base directory containing grey matter files.
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **path**: Absolute path to the base directory containing white matter files.
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `path` and `glob_name_pattern` fields for each segment in the `segments_dict`. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.
- < *_name_pattern > variables do not need a leading slash ("/"). This is already accounted for. 

---

In [20]:
base_directory = r'/Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/vbm'
grey_matter_glob_name_pattern = '*/mri/*mwp1*resampled*'
white_matter_glob_name_pattern = '*/mri/*mwp2*resampled*'
csf_glob_name_pattern = '*/mri/*mwp3*resampled*'

In [21]:
from calvin_vbm.file_utils.import_matrices import import_matrices_from_folder #<----- CALVIN IMPORT

def import_dataframes_from_folders(base_directory, grey_matter_glob_name_pattern, white_matter_glob_name_pattern, csf_glob_name_pattern):
    """
    Imports dataframes from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - grey_matter_glob_name_pattern (str): Glob pattern for grey matter data.
    - white_matter_glob_name_pattern (str): Glob pattern for white matter data.
    - csf_glob_name_pattern (str): Glob pattern for cerebrospinal fluid data.
    
    Returns:
    - dict: A dictionary containing dataframes for grey matter, white matter, and cerebrospinal fluid.
    """
    

    segments_dict = {
        'grey_matter': {'path': base_directory, 'glob_name_pattern': grey_matter_glob_name_pattern},
        'white_matter': {'path': base_directory, 'glob_name_pattern': white_matter_glob_name_pattern},
        'cerebrospinal_fluid': {'path': base_directory, 'glob_name_pattern': csf_glob_name_pattern}
    }

    dataframes_dict = {}

    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=v['path'], file_pattern=v['glob_name_pattern'])
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'These are the filenames per subject {dataframes_dict[k].columns}')
        print('--------------------------------')

    return dataframes_dict


In [22]:
dataframes_dict = import_dataframes_from_folders(base_directory, grey_matter_glob_name_pattern, white_matter_glob_name_pattern, csf_glob_name_pattern)

I will search:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/vbm/*/mri/*mwp1*resampled*
Imported data grey_matter data with 902629 voxels and 50 patients
These are the filenames per subject Index(['mwp10001_resampled.nii', 'mwp10002_resampled.nii',
       'mwp10003_resampled.nii', 'mwp10004_resampled.nii',
       'mwp10005_resampled.nii', 'mwp10006_resampled.nii',
       'mwp10007_resampled.nii', 'mwp10008_resampled.nii',
       'mwp10009_resampled.nii', 'mwp10010_resampled.nii',
       'mwp10011_resampled.nii', 'mwp10012_resampled.nii',
       'mwp10013_resampled.nii', 'mwp10014_resampled.nii',
       'mwp10015_resampled.nii', 'mwp10016_resampled.nii',
       'mwp10017_resampled.nii', 'mwp10018_resampled.nii',
       'mwp10019_resampled.nii', 'mwp10020_resampled.nii',
       'mwp10021_resampled.nii', 'mwp10022_resampled.nii',
       'mwp10023_resampled.nii', 'mwp10024_resampled.nii',
       'mwp10025_resampled.nii', 'mwp10026_resampled.nii',
       'mwp10027_resampl

**Extract Subject ID From File Names**
- Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. If nothing preceding subject identifier, enter "".
- Do NOT include mwp[1/2/3] in this. 
2) Proceeding the subject ID. If nothing proceeding subject identifier, enter "".

In [23]:
preceding_id = ''
proceeding_id = '_re'

In [24]:
import re

def remove_specific_mwp_integer_pattern(text):
    # Define the pattern to search for: 'mwp' followed by [1], [2], or [3]
    pattern = r'mwp[123]'
    # Replace the first occurrence of the pattern with an empty string
    return re.sub(pattern, '', text, count=1)


def extract_and_rename_subject_id(dataframe, split_command_dict):
    """
    Renames the columns of a dataframe based on specified split commands.

    Parameters:
    - dataframe (pd.DataFrame): The dataframe whose columns need to be renamed.
    - split_command_dict (dict): A dictionary where the key is the split string 
                                 and the value is the order to take after splitting 
                                 (0 for before the split, 1 for after the split, etc.).

    Returns:
    - pd.DataFrame: Dataframe with renamed columns.

    Example:
    >>> data = {'subject_001': [1, 2, 3], 'patient_002': [4, 5, 6], 'control_003': [7, 8, 9]}
    >>> df = pd.DataFrame(data)
    >>> split_commands = {'_': 1}
    >>> new_df = extract_and_rename_subject_id(df, split_commands)
    >>> print(new_df.columns)
    Index(['001', '002', '003'], dtype='object')
    """

    raw_names = dataframe.columns
    name_mapping = {}

    # For each column name in the dataframe
    for name in raw_names:
        new_name = name  # Default to the original name in case it doesn't match any split command

        # Check each split command to see if it applies to this column name
        for k, v in split_command_dict.items():
            if k in new_name:
                new_name = remove_specific_mwp_integer_pattern(new_name)
                if k !='':
                    new_name = new_name.split(k)[v]
        # Add the original and new name to the mapping
        name_mapping[name] = new_name

    # Rename columns in the dataframe based on the mapping
    return dataframe.rename(columns=name_mapping)

def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """
    
    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


In [25]:
renamed_dfs = rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id)

Dataframe:  grey_matter


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  white_matter


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  cerebrospinal_fluid


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Import Control Segments

In [26]:
base_directory_control = '/Volumes/OneTouch/datasets/adni/neuroimaging/true_control/anat/mri'
control_grey_matter_glob_name_pattern = '*smwp1*resampled*'
control_white_matter_glob_name_pattern = '*smwp2*resampled*'
control_csf_glob_name_pattern = '*smwp3*resampled*'

In [27]:
from calvin_vbm.file_utils.import_matrices import import_matrices_from_folder
def import_control_dataframes(base_directory, control_grey_matter_glob_name_pattern, control_white_matter_glob_name_pattern, control_csf_glob_name_pattern):
    """
    Imports control dataframes from specified directories and glob name patterns.

    Parameters:
    - base_directory (str): The base directory where the data resides.
    - control_grey_matter_glob_name_pattern (str): Glob pattern for grey matter data.
    - control_white_matter_glob_name_pattern (str): Glob pattern for white matter data.
    - control_csf_glob_name_pattern (str): Glob pattern for cerebrospinal fluid data.

    Returns:
    - dict: A dictionary containing control dataframes for grey matter, white matter, and cerebrospinal fluid.
    """
    
    segments_dict = {
        'grey_matter': {'path': base_directory, 'glob_name_pattern': control_grey_matter_glob_name_pattern},
        'white_matter': {'path': base_directory, 'glob_name_pattern': control_white_matter_glob_name_pattern},
        'cerebrospinal_fluid': {'path': base_directory, 'glob_name_pattern': control_csf_glob_name_pattern}
    }

    control_dataframes_dict = {}
    for k, v in segments_dict.items():
        control_dataframes_dict[k] = import_matrices_from_folder(connectivity_path=v['path'], file_pattern=v['glob_name_pattern']);
        print(f'Imported data {k} data with {control_dataframes_dict[k].shape[0]} voxels and {control_dataframes_dict[k].shape[1]} patients')
        print(f'Example subject filename: {control_dataframes_dict[k].columns[-1]}')
        print('--------------------------------')

    return control_dataframes_dict


In [28]:
control_dataframes_dict = import_control_dataframes(base_directory_control, control_grey_matter_glob_name_pattern, control_white_matter_glob_name_pattern, control_csf_glob_name_pattern)

I will search:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_control/anat/mri/*smwp1*resampled*
Imported data grey_matter data with 902629 voxels and 136 patients
Example subject filename: smwp1002_S_4264_resampled.nii
--------------------------------
I will search:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_control/anat/mri/*smwp2*resampled*
Imported data white_matter data with 902629 voxels and 136 patients
Example subject filename: smwp2941_S_4376_resampled.nii
--------------------------------
I will search:  /Volumes/OneTouch/datasets/adni/neuroimaging/true_control/anat/mri/*smwp3*resampled*
Imported data cerebrospinal_fluid data with 902629 voxels and 136 patients
Example subject filename: smwp3941_S_4376_resampled.nii
--------------------------------


# Generate Z-Scored Atrophy Maps for Each Segment

In [29]:
import pandas as pd
from typing import Tuple
def threshold_probabilities(patient_df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    patient_df = patient_df.where(patient_df > threshold, 0)
    return patient_df

def calculate_z_scores(control_df: pd.DataFrame, patient_df: pd.DataFrame, matter_type=None) -> pd.DataFrame:
    """
    Function to calculate voxel-wise mean, standard deviation for control group and z-scores for patient group.

    Args:
    control_df (pd.DataFrame): DataFrame where each column represents a control subject, 
                               and each row represents flattened image data for a voxel.
    patient_df (pd.DataFrame): DataFrame where each column represents a patient, 
                               and each row represents flattened image data for a voxel.

    Returns:
    patient_z_scores (pd.DataFrame): DataFrame of voxel-wise z-scores calculated for each patient using control mean and std.
    """

    # # Mask the dataframes to only consider tissues over acceptable probability thresholds
    # # Using p>0.2, as typical masking to MNI152 segments uses P > 0.2 for a given segment
    
    # # Now you can use the function to apply a threshold to patient_df and control_df
    threshold = 0.2
    patient_df = threshold_probabilities(patient_df, threshold)
    control_df = threshold_probabilities(control_df, threshold)

    # Calculate mean and standard deviation for each voxel in control group
    control_mean = control_df.mean(axis=1)
    control_std = control_df.std(axis=1)

    # Initialize DataFrame to store patient z-scores
    patient_z_scores = pd.DataFrame()

    # Calculate z-scores for each patient using control mean and std
    for patient in patient_df.columns:
        patient_z_scores[patient] = (patient_df[patient] - control_mean) / control_std

    return patient_z_scores

In [30]:
def process_atrophy_dataframes(dataframes_dict, control_dataframes_dict):
    """
    Processes the provided dataframes to calculate z-scores and determine significant atrophy.

    Parameters:
    - dataframes_dict (dict): Dictionary containing patient dataframes.
    - control_dataframes_dict (dict): Dictionary containing control dataframes.

    Returns:
    - tuple: A tuple containing two dictionaries - atrophy_dataframes_dict and significant_atrophy_dataframes_dict.
    """
    
    atrophy_dataframes_dict = {}
    significant_atrophy_dataframes_dict = {}

    for k in dataframes_dict.keys():
        atrophy_dataframes_dict[k] = calculate_z_scores(control_df=control_dataframes_dict[k], patient_df=dataframes_dict[k])
        if k == 'cerebrospinal_fluid':
            significant_atrophy_dataframes_dict[k] = atrophy_dataframes_dict[k].where(atrophy_dataframes_dict[k] > 2, 0)
        else:
            significant_atrophy_dataframes_dict[k] = atrophy_dataframes_dict[k].where(atrophy_dataframes_dict[k] < -2, 0)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return atrophy_dataframes_dict, significant_atrophy_dataframes_dict


In [31]:
unthresholded_atrophy_dataframes_dict, significant_atrophy_dataframes_dict = process_atrophy_dataframes(dataframes_dict, control_dataframes_dict)

Dataframe:  grey_matter


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  white_matter


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  cerebrospinal_fluid


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



**Add the Significant Atrophy Maps Together**

In [34]:
import pandas as pd
from itertools import combinations

def generate_composite_maps(dataframes_dict):
    """
    Combines the absolute values of DataFrames for all possible combinations of keys
    in the dictionary, starting from pairs up to the full set of keys, if applicable. 
    Each resulting DataFrame is added back to the dictionary with a key formatted as 
    'K1+K2+...+Kn', where K1, K2, ..., Kn are the original keys of the combined DataFrames.

    Parameters:
    - dataframes_dict (dict): A dictionary where each key is a string representing the DataFrame's name, 
                              and each value is a pandas DataFrame.
    
    Returns:
    - dict: The original dictionary updated with additional keys for each combination of DataFrames. 
            Each new key contains the sum of absolute values of the combined DataFrames.
    """
    
    keys = list(dataframes_dict.keys())
    for r in range(2, len(keys)+1):  # Create combinations for r-tuples where r ranges from 2 to len(keys)
        for key_tuple in combinations(keys, r):
            combined_key = '+'.join(key_tuple)
            combined_df = pd.DataFrame()

            for k in key_tuple:
                abs_df = dataframes_dict[k].abs()  # Take the absolute value of each DataFrame in the combination

                if combined_df.empty:  # If the combined_df is still empty, initialize it with the first absolute DataFrame
                    combined_df = abs_df.copy()
                else:
                    combined_df += abs_df  # Add the absolute values to the combined DataFrame

            dataframes_dict[combined_key] = combined_df  # Store the combined DataFrame in the dictionary under the new key
            
    return dataframes_dict


In [33]:
thresholded_atrophy_dataframes_dict = generate_composite_maps(significant_atrophy_dataframes_dict)

**Save the Atrophy Results**

Save Raw Z-Scores

In [35]:
import os
from calvin_vbm.nifti_utils.generate_nifti import view_and_save_nifti #<-----CAlVIN IMPORT
from tqdm import tqdm

def save_nifti_to_bids(dataframes_dict, bids_base_dir, analysis='tissue_segment_z_scores', ses=None, dry_run=True):
    """
    Saves NIFTI images to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the NIFTI 
    images accordingly. The function currently has the view_and_save_nifti call commented out 
    for safety. Uncomment this call if you wish to actually save the NIFTI images.
    
    Example:
    >>> dfs = { ... }  # some dictionary with dataframes
    >>> save_nifti_to_bids(dfs, '/path/to/base/dir')
    """
    
    for k in tqdm(dataframes_dict.keys()):
        for col in dataframes_dict[k].columns:
            
            # Define BIDS Directory Architecture
            sub_no = col
            if ses is None:
                ses_no = '01'
            else:
                ses_no = ses
            
            # Define and Initialize the Save Directory
            out_dir = os.path.join(bids_base_dir, f'sub-{sub_no}', f'ses-{ses_no}', analysis)
            os.makedirs(out_dir, exist_ok=True)
            
            # Save Image to BIDS Directory
            if dry_run:
                print(out_dir+f'/sub-{sub_no}_{k}')
            else:
                view_and_save_nifti(matrix=dataframes_dict[k][col],
                                    out_dir=out_dir,
                                    output_name=(f'sub-{sub_no}_{k}'))


# Save the Z-Scored Maps

Where to save?

In [38]:
base_directory = '/Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds'

Unthresholded Maps

In [ ]:
save_nifti_to_bids(unthresholded_atrophy_dataframes_dict, bids_base_dir=base_directory, analysis='unthresholded_tissue_segment_z_scores', dry_run=False);

Thresholded Maps - The 'Real' Atrophy


In [39]:
save_nifti_to_bids(thresholded_atrophy_dataframes_dict, bids_base_dir=base_directory, analysis='thresholded_tissue_segment_z_scores', dry_run=False);

  0%|          | 0/7 [00:00<?, ?it/s]

Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0001/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0002/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0003/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0004/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0005/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0006/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 

 14%|█▍        | 1/7 [00:50<05:04, 50.78s/it]

Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0001/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0002/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0003/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0004/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0005/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0006/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 

 29%|██▊       | 2/7 [01:40<04:09, 49.98s/it]

...

 71%|███████▏  | 5/7 [04:00<01:33, 46.81s/it]

Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0001/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0002/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0003/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0004/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0005/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0006/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 

 86%|████████▌ | 6/7 [04:45<00:46, 46.28s/it]

Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0001/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0002/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0003/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0004/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0005/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 
 /Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/atrophy_seeds/z_atrophy_seeds/sub-0006/ses-01/thresholded_tissue_segment_z_scores
Image saved to: 

100%|██████████| 7/7 [05:33<00:00, 47.59s/it]


All Done. Enjoy your atrophy seeds.

--Calvin